In [1]:
pip install pandas pyarrow fsspec s3fs


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install lakefs-s3


ERROR: Could not find a version that satisfies the requirement lakefs-s3 (from versions: none)
ERROR: No matching distribution found for lakefs-s3

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
import requests
import pandas as pd
from datetime import datetime
import pytz

# API endpoint and parameters
WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "1de9eaf371dc114af656a95cd26febd8"  # Replace with your actual API key

provinces = {
    "Bangkok":{
            "lat": 13.7367,
            "lon": 100.5232
    },
    "Satitram Association":{
        "lat": 13.75291,
        "lon": 100.618616
    },

}
# Function to fetch and process weather data
def get_weather_data(province='Pathum Thani'):
    
    params = {
        "lat": provinces[province]['lat'],
        "lon": provinces[province]['lon'],
        "appid": API_KEY,
        "units": "metric"
    }
    try:
        # Make API request
        response = requests.get(WEATHER_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        
        # Convert timestamp to datetime
        # created_at = datetime.fromtimestamp(data['dt'])

        dt = datetime.now()
        thai_tz = pytz.timezone('Asia/Bangkok')
        created_at = dt.replace(tzinfo=thai_tz)


        timestamp = datetime.now()
        
        # Create dictionary with required fields
        weather_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'created_at': created_at,
            'requested_province':province,
            'location': data['name'],
            'weather_main': data['weather'][0]['main'],
            'weather_description': data['weather'][0]['description'],
            'main.temp': data['main']['temp']
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return weather_dict
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None

In [36]:
df=pd.DataFrame([get_weather_data(p) for p in list(provinces.keys())])
df.info()
df.head()

06:16:51.546 | INFO    | Task run 'get_weather_data' - Created task run 'get_weather_data' for task 'get_weather_data'

06:16:52.135 | ERROR   | Task run 'get_weather_data' - Task run failed with exception: TypeError("string indices must be integers, not 'str'") - Retries are exhausted
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/prefect/task_engine.py", line 790, in run_context
    yield self
  File "/usr/local/lib/python3.11/site-packages/prefect/task_engine.py", line 1393, in run_task_sync
    engine.call_task_fn(txn)
  File "/usr/local/lib/python3.11/site-packages/prefect/task_engine.py", line 818, in call_task_fn
    result = call_with_parameters(self.task.fn, parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/prefect/utilities/callables.py", line 208, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_48/1917810910.py", line 17, in get_weather_data
    location = location_context['location']
               ~~~~~~~~~~~~~~~~^^^^^^^^^^^^
TypeError: string indices must be integers, not 'str'

06:16:52.254 | ERROR   | Task run 'get_weather_data' - Finished in state Failed("Task run encountered an exception TypeError: string indices must be integers, not 'str'")

TypeError: string indices must be integers, not 'str'

In [ ]:
pip install lakefs-spec


In [4]:

dt = datetime.now()
thai_tz = pytz.timezone('Asia/Bangkok')
dt = dt.replace(tzinfo=thai_tz)
print(dt) 

2025-05-12 17:50:32.503050+06:42


In [31]:
 # lakeFS credentials from your docker-compose.yml
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"
    
# lakeFS endpoint (running locally)
lakefs_endpoint = "http://lakefs-dev:8000/"
    
# lakeFS repository, branch, and file path
repo = "weather"
branch = "main"
path = "weather.parquet"

# Construct the full lakeFS S3-compatible path
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"
    
# Configure storage_options for lakeFS (S3-compatible)
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint   
    }   
}
df.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,    
    partition_cols=['year','month','day','hour'],
    
)


# test read parquet files

In [32]:
path_all_partition = 's3a://weather3/main/weather.parquet'

df2=pd.read_parquet(    
    path=path_all_partition,
    storage_options=storage_options
)
df2.info()
df2.head()

FileNotFoundError: weather3/main/weather.parquet

In [12]:
path_single_partition = 's3a://weather3/main/weather.parquet/year=2025/month=4/day=10/hour=7/70cf855c5c5e4659ae01aba885c731c3-0.parquet'

df2=pd.read_parquet(    
    path=path_single_partition,
    storage_options=storage_options,
)
df2.info()
df2.head()

FileNotFoundError: weather3/main/weather.parquet/year=2025/month=4/day=10/hour=7/70cf855c5c5e4659ae01aba885c731c3-0.parquet

In [91]:
print(dt)

2025-04-10 07:50:44.311326+06:42


# Test Duck and Dask

In [88]:
# pip install duckdb
import duckdb
import pandas as pd

# Connect to an in-memory DuckDB instance.
con = duckdb.connect(database=':memory:')

storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}

# Use DuckDB's read_parquet() function to read the dataset.
# Assume the dataset is stored in the directory "output_parquet_dataset" with hive partitions:
# output_parquet_dataset/year=2025/month=02/day=17/...
query = """
INSTALL httpfs;
LOAD httpfs;

SET s3_endpoint='lakefs-dev:8000';
SET s3_access_key_id='access_key'; 
SET s3_secret_access_key='secret_key'; 
SET s3_url_style='path';
SET s3_use_ssl=false;

SELECT *
FROM read_parquet('s3a://weather3/main/weather.parquet')
"""

df_duck = con.execute(query).df()

print("DuckDB Parquet Query Result:")
df_duck.info()
df_duck.head(20)

HTTPException: HTTP Error: Unable to connect to URL "http://lakefs-dev:8000/weather3/main/weather.parquet": 404 (Not Found).

In [89]:
# pip install dask
import dask.dataframe as dd
df2 = dd.read_parquet(
    path=path_all_partition,
    storage_options=storage_options,
    dtype_backend='pyarrow'
)  

ImportError: An error occurred while calling the read_parquet method registered to the pandas backend.
Original Message: pyarrow>=10.0.1 is required for PyArrow backed StringArray.